In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, TensorDataset

from typing import List, Tuple, NoReturn, Any, Optional, Union
import pandas as pd
import numpy as np

from datasets import (
    Dataset,
    load_from_disk,
    concatenate_datasets,
)

from tqdm.auto import tqdm, trange
import os

import datasets
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, HfArgumentParser

In [3]:
dataset = load_from_disk("/opt/ml/data/train_dataset")
train_dataset = dataset["train"]
valid_dataset = dataset["validation"]
dataset

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 240
    })
})

In [4]:
model_checkpoint = "klue/roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [5]:
df = pd.read_csv("/opt/ml/data/bm25/bm25_valid.csv", index_col = 0)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/opt/ml/data/bm25/bm25_valid.csv'

In [9]:
len(answers), len(question), len(original_context)

(240, 240, 240)

In [10]:
for idx, passages in enumerate(df.context.tolist()):
    p_list = passages.split('▦')
    
    if original_context[idx] in p_list:
        top_k_passage.append(p_list)
        target_index = p_list.index(original_context[idx])
        target.append(target_index)
    else :
        p_list = [original_context[idx]] + p_list[:-1]
        top_k_passage.append(p_list)
        target.append(0)

In [11]:
top_k_passage[0]

['순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다.\n\n경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경영인에 대한 최초의 배상 판결을 했다.  \n\n2004년 10월 신행정수도의건설을위한특별조치법 위헌 확인 소송에서 9인의 재판관 중 유일하게 각하 견해를 내었다. 소수의견에서 전효숙 재판관은 다수견해의 문제점을 지적하면서 관습헌법 법리를 부정하였다. 전효숙 재판관은 서울대학교 근대법학교육 백주년 기념관에서 열린 강연에서, 국회가 고도의 정치적인 사안을 정치로 풀기보다는 헌법재판소에 무조건 맡겨서 해결하려는 자세는 헌법재판소에게 부담스럽다며 소회를 밝힌 바 있다.',
 '순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다.\\n\\n경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경

In [79]:
len(top_k_passage), len(target)

(240, 240)

In [80]:
answers

["{'answer_start': [284], 'text': ['한보철강']}",
 "{'answer_start': [146], 'text': ['1871년']}",
 "{'answer_start': [517], 'text': ['나뭇잎']}",
 "{'answer_start': [1109], 'text': ['금대야']}",
 "{'answer_start': [386], 'text': ['수평적 관계']}",
 "{'answer_start': [408], 'text': ['옥음방송']}",
 "{'answer_start': [192], 'text': ['코칭 스티치']}",
 "{'answer_start': [31], 'text': ['복잡한 감염병']}",
 "{'answer_start': [625], 'text': ['스페인']}",
 "{'answer_start': [71], 'text': ['20세기 초']}",
 '{\'answer_start\': [274], \'text\': [\'"5월의 왕"\']}',
 '{\'answer_start\': [901], \'text\': ["\'일급 비밀 프로젝트 2501\'"]}',
 "{'answer_start': [430], 'text': ['테헤란']}",
 "{'answer_start': [317], 'text': ['역사교육과정개발추진위원회']}",
 "{'answer_start': [1429], 'text': ['1967년 11월 15일']}",
 "{'answer_start': [0], 'text': ['1965년']}",
 "{'answer_start': [162], 'text': ['아리크 부케']}",
 "{'answer_start': [709], 'text': ['〈중앙일보〉']}",
 "{'answer_start': [304], 'text': ['미타케성']}",
 "{'answer_start': [536], 'text': ['전체 4순위']}",
 "{'answer_start': [689

In [82]:
custom_dataset = {}

In [83]:
custom_dataset["answers"] = answers
custom_dataset["question"] = question
custom_dataset["original_context"] = original_context
custom_dataset["top_k_passage"] = top_k_passage
custom_dataset["target"] = target

In [2]:
import sys
sys.path

['/opt/ml/mrc-level2-nlp-02/retriever',
 '/opt/ml/.vscode-server/extensions/ms-toolsai.jupyter-2021.9.1101343141/pythonFiles',
 '/opt/ml/.vscode-server/extensions/ms-toolsai.jupyter-2021.9.1101343141/pythonFiles/lib/python',
 '/opt/conda/lib/python38.zip',
 '/opt/conda/lib/python3.8',
 '/opt/conda/lib/python3.8/lib-dynload',
 '',
 '/opt/conda/lib/python3.8/site-packages',
 '/opt/conda/lib/python3.8/site-packages/IPython/extensions',
 '/opt/ml/.ipython']